# Demo Scenario 2
> Validate Demonstrate Scenario 2 on a notebook

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| eval: false

from llmcam.downloader import *
from llmcam.dtcam import *
from llmcam.file_manager import *
from llmcam.fn_to_fc import *
from llmcam.store import *
from llmcam.yolo import *
from llmcam.plotting import *
from llmcam.gpt4v import *

In [ ]:
#| eval: false

def send_notification(
        msg: str  # The message to send
    ):
    """Send a notification"""
    global messages
    messages.append(form_msg("assistant", f"Notification: {msg}"))
    complete(messages, tools)
    return msg

from threading import Thread
import time

def start_notifications_stream(
    messages: list  # Previous conversation with the user
):
    """Start the notifications stream. At the end of the stream, notify users about the results."""
    global tools

    # Add sending notification to tool schema
    subtools = [ tool for tool in tools if tool['function']['name'] != 'start_notifications_stream' ]
    subtools.append(tool_schema(send_notification, 'send_notification'))

    # Copy the messages to avoid modifying the original list
    messages = [ message for message in messages ]

    # Define the functionality to use
    def monitoring_loop():
        for _ in range(5):  
            time.sleep(10)  

            # Ask GPT if the condition is met
            messages.append({
                "role": "system",
                "content": "Evaluate the data. Does it satisfy the condition stated by the user? When the condition is met, send a message to user. "
            })
            complete(messages, tools=subtools)

    # Start the notifications stream
    t = Thread(target=monitoring_loop)
    t.start()

    return 'Notifications stream started'

In [ ]:
#| eval: false
tools = []
def execute_handler(function_name, **kwargs):
    execute_handler_core(tools, function_name, **kwargs)

tools.extend([handler_schema(function, service_name="toolbox_handler", fixup=execute_handler) for function in [
        add_api_tools,
        add_function_tools,
        remove_tools,
        capture_youtube_live_frame_and_save,
        ask_gpt4v_about_image_file,
        list_image_files,
        plot_object
    ]])

tools.append(tool_schema(start_notifications_stream, 'notification'))
tools[-1]['function']['parameters'] = {
    'type': 'object',
    'properties': {
        'messages': {
            'type': 'array',
            'items': {
                'type': 'object',
                'properties': {
                    'role': {
                        'type': 'string',
                        'enum': ['user', 'tool', 'system', 'assistant']
                    },
                    'content': {
                        'type': 'string'
                    }
                }
            }
        }
    }
}

In [ ]:
#| eval: false

messages = form_msgs([
    ("system", "You are a helpful system administrator. Use the supplied tools to help the user."),
    ("user", "What can LLMCAM do?")
])
# messages.append(form_msg("user", "Here is the live feed for Santa Claus Village: https://www.youtube.com/watch?v=Cp4RRAEgpeU to capture"))
# messages.append(form_msg("user", "How many people are there currently?"))
# messages.append(form_msg("user", "List the images captured at Santa Claus Village during today"))
# messages.append(form_msg("user", "Plot the number of people in those images using your tools."))
# messages.append(form_msg("user", "Notify me when there are fewer than 10 people."))
complete(messages, tools)
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
What can LLMCAM do?
>> Assistant:
LLMCAM is designed to capture and analyze images from YouTube Live streams. It provides
functionalities such as:  1. **Capture a Frame**: You can capture a JPEG image from a YouTube Live
stream and save it in a specific directory.  2. **List Captured Images**: Displays all captured
image files, which follow a specific naming scheme.  3. **Ask About Image**: Utilize AI to analyze
and provide insights about the quantitative information in a given image file.  These features allow
you to monitor live streams and extract relevant visual data efficiently.
>> User:
Here is the live feed for Santa Claus Village: https://www.youtube.com/watch?v=Cp4RRAEgpeU to
capture
>> Assistant:
I've captured an image from the YouTube Live feed of Santa Claus Village. If you need further
analysis or have any other requests, feel free to ask!
>> User:
How many people are there

In [ ]:
#| eval: false
print_msgs(messages)

>> System:
You are a helpful system administrator. Use the supplied tools to help the user.
>> User:
What can LLMCAM do?
>> Assistant:
LLMCAM is designed to capture and analyze images from YouTube Live streams. It provides
functionalities such as:  1. **Capture a Frame**: You can capture a JPEG image from a YouTube Live
stream and save it in a specific directory.  2. **List Captured Images**: Displays all captured
image files, which follow a specific naming scheme.  3. **Ask About Image**: Utilize AI to analyze
and provide insights about the quantitative information in a given image file.  These features allow
you to monitor live streams and extract relevant visual data efficiently.
>> User:
Here is the live feed for Santa Claus Village: https://www.youtube.com/watch?v=Cp4RRAEgpeU to
capture
>> Assistant:
I've captured an image from the YouTube Live feed of Santa Claus Village. If you need further
analysis or have any other requests, feel free to ask!
>> User:
How many people are there

1. User: What Function Calls (FCs) do you have?
2. Assistant: I can monitor YouTube Live feeds and integrate with various services to provide insights and notifications.
3. User: Here’s the YouTube Live feed for Santa Claus Village.
4. Assistant: The YouTube Live feed for Santa Claus Village has been registered.
5. User: How many people are currently there?
6. Assistant: There are currently 52 people at Santa Claus Village.
7. User: Plot the change in the number of people at Santa Claus Village over the last 3 hours.
8. Assistant: [Generates and displays a bar chart showing the trend in people count over the last 3 hours.]
9. User: Notify me if there are fewer than 10 people.
10. Assistant: Notification rule set. I’ll alert you when the number of people at Santa Claus Village drops below 10.

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()